Esteban Schmitt   
CS559  
Assignment 1 - Linear Regression   
I pledge my honor that I have abided by the Stevens honor system

Import and gather the data we are using from ucirepo 

In [10]:
import numpy as np
import pandas as pd

from mlxtend.plotting import scatterplotmatrix

import matplotlib.pyplot as plt

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
bike_sharing = fetch_ucirepo(id=275) 
  

# data (as pandas dataframes) 
X = bike_sharing.data.features 
y = bike_sharing.data.targets
 
  
# metadata 
X.head()
  
# variable information 
#print(y)

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0


We want to get rid of the dateday column as we already have the season which is the more important feature. We also convert the temperature into farenheit increase variance in temperature, as that will be one of the features that matters most

In [11]:
#drop the date
X = X.drop('dteday',axis = 1)
#drop the temperature as we already have feels like temp
X = X.drop('temp',axis=1)
#convert to farenheit to account for greater variance

atemp = X['atemp']
newtemp = atemp.apply(lambda a: (a * (9/5)) + 32) #conversion to Farenheit
X['atemp'] = newtemp
X.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,atemp,hum,windspeed
0,1,0,1,0,0,6,0,1,32.51822,0.81,0.0
1,1,0,1,1,0,6,0,1,32.49086,0.80,0.0
2,1,0,1,2,0,6,0,1,32.49086,0.80,0.0
3,1,0,1,3,0,6,0,1,32.51822,0.75,0.0
4,1,0,1,4,0,6,0,1,32.51822,0.75,0.0


Regularize the data

In [12]:
def standardize(X_train, X_test):
    # Calculate the mean and standard deviation of the training set
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)

    # To prevent division by zero, set std=1 where std is zero
    std[std == 0] = 1

    # Standardize the training set
    X_train_scaled = (X_train - mean) / std

    # Use the training set's mean and std to standardize the test set
    X_test_scaled = (X_test - mean) / std

    return X_train_scaled, X_test_scaled

Create the mini-batch gradient descent function


In [13]:
# MBGD implementation
def ridge_mbgd(X, y, learning_rate=0.01, n_iterations=1000,batch_size = 32):
    m, n = X.shape
    w = np.random.randn(n, 1)  # Initialize w with random values
    y = y.values.reshape(-1, 1)    # Reshape y to be a column vector
    mse_history = []  # List to store MSE values over iterations

    for iteration in range(n_iterations):
        
        for i in range(64): # 
            random_index = np.random.randint(m-batch_size-1) # we don't want an overflow
            end_range = random_index + batch_size

            Xi = X[random_index:end_range+1]
            yi = y[random_index:end_range+1]

           

            gradients = 2 * Xi.T.dot(Xi.dot(w) - yi) + learning_rate*w.T * w # Ridge regression L2 Regularization
            

            w = w - learning_rate * gradients

        # Store MSE for every iteration
        mse = np.mean((X.dot(w) - y) ** 2)
        mse_history.append(mse)

        # if iteration % 100 == 0 and iteration != 0:
        #     print(f"Iteration {iteration}: MSE = {mse}")

    return w, mse_history

We make a function to split our data into k fold and k-1 folds


In [14]:
def k_fold_test_train_split(X,y,test_size=0.3,startIndex = 0,k=10):
    
    num_data_points = X.shape[0]

    shuffled_indices = np.random.permutation(num_data_points)
    
    fold_size = num_data_points // k # k is the number of folds

    test_indices = shuffled_indices[startIndex:startIndex+fold_size]


    
    if startIndex == 0:
        train_indices = shuffled_indices[startIndex+fold_size:] # the first fold
    elif startIndex == len(shuffled_indices) - fold_size:
        train_indices = shuffled_indices[0:len(shuffled_indices)-fold_size] # the last fold
    else:
        train_indices = np.concatenate((shuffled_indices[0:startIndex],shuffled_indices[startIndex+fold_size:])) # folds in between first and last



    X_train = X.iloc[train_indices]
    X_test = X.iloc[test_indices]
    y_train = y.iloc[train_indices]
    y_test = y.iloc[test_indices]

    return X_train, X_test, y_train, y_test




Loop through different test and train sets and cross validate the mini-batch descent algorithm

We chose 32 as our batch size as per the article "REVISITING SMALL BATCH TRAINING FOR
DEEP NEURAL NETWORKS" by Dominic Masters and Carlo Luschi

In [15]:
def k_means(X,y,test_size = 0.3,k_size = 10):
    mse_validate = []
    fold_size = len(y) // k_size

    for i in range(0,10):
        

        X_train, X_test, y_train, y_test = k_fold_test_train_split(X, y, test_size=0.3,startIndex=i * fold_size )

        # standardize the testing data
        X_train_scaled, X_test_scaled = standardize(X_train.values, X_test.values)

        # Add intercept term to the standardized data
        X_train_intercept_scaled = np.column_stack([np.ones(X_train_scaled.shape[0]), X_train_scaled])
        X_test_intercept_scaled = np.column_stack([np.ones(X_test_scaled.shape[0]), X_test_scaled])


        
        # Get the value of w using mini-batch gradient descent
        w_mbgd, mse_history = ridge_mbgd(X_train_intercept_scaled, y_train, learning_rate=0.0006, n_iterations=600,batch_size=32)
        # testing showed more optimal results with 600 iterations and learning rate of 0.0006

        y_pred_mbgd = np.dot(X_test_intercept_scaled, w_mbgd)
        # Mean Squared Error (MSE)
        mse_mbgd = np.mean((y_pred_mbgd - y_test.values)**2)
        mse_validate.append(mse_mbgd)
        
        print(f"MBGD Mean Squared Error: {mse_mbgd}")
    return mse_validate

mse_validate = k_means(X,y,test_size=0.3,k_size = 10)

MBGD Mean Squared Error: 20770.899866881795
MBGD Mean Squared Error: 19903.110299665033
MBGD Mean Squared Error: 18851.333242985118
MBGD Mean Squared Error: 20314.339303052984
MBGD Mean Squared Error: 19829.264804430877
MBGD Mean Squared Error: 21057.08574786707
MBGD Mean Squared Error: 20925.13269236615
MBGD Mean Squared Error: 19442.892990141565
MBGD Mean Squared Error: 22277.739894098
MBGD Mean Squared Error: 21829.257871750815


Calculate the average MSE

In [16]:
avg_mse = sum(mse_validate) / len(mse_validate)

print(f"Average MSE is: {avg_mse}" )

Average MSE is: 20520.105671323934
